<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/quick_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
!pip install -U git+https://github.com/graphistry/pygraphistry.git@cudf #-cat-final
!pip install -U git+https://github.com/graphistry/cu-cat.git  # # @ #cu_cat_regpt

In [ ]:
import cuml,cudf
cuml.__version__

'22.10.00a+119.gb30ef85c1'

In [ ]:
!echo "0.01.0" > VERSION.txt
# !cp VERSION.txt /root/.local/lib/python3.9/site-packages/cu_cat/
!cp VERSION.txt /usr/local/lib/python3.9/dist-packages/cu_cat/

In [ ]:
# from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

/usr/local/lib/python3.9/dist-packages/cu_cat/__init__.py


In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import cudf
from time import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 200)

In [ ]:
import graphistry

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key

# graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__


'0.28.7+422.g19e9f6c'

In [ ]:
!pip install dirty_cat

In [ ]:
!nvidia-smi

Thu Apr 20 06:30:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W /  70W |    103MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# run independently

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
X=1

In [ ]:
# bench=pd.DataFrame(columns=['dataset','row','col','cc_time','dc_time'])
# !wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet

# winlogsA=pd.read_parquet('part.88.parquet')
# n=[1000,5000,10000,50000,100000] #,1000000]
# for p in np.arange(X):

  # for ii,i in enumerate(n):

winlogs=winlogsA.sample(5000,replace=False)#.iloc[:,:3]
winlogs=winlogs.dropna(how='any',axis=1)
# winlogs.shape
g = graphistry.nodes(cudf.from_pandas(winlogs))

t=time()
# for jj,j in enumerate(['cu_cat','dirty_cat']):
g.featurize(feature_engine='dirty_cat',memoize=False)
t1=time()-t

#     g = graphistry.nodes(winlogs)
#     t=time()
#     # for jj,j in enumerate(['cu_cat','']):
#     g.featurize(feature_engine='dirty_cat',memoize=False)
#     t2=time()-t
    
#     bench.loc[1+bench.shape[0]]=['winlogs',i,winlogs.shape[1],t1,t2]
#     bench.to_csv('winlogs_bench.txt',sep='\t',mode='a')
# !cp *.txt /content/drive/MyDrive/graphistry

Featurizing nodes with feature_engine=dirty_cat
Using GPU: dirty_cat


AttributeError: ignored

In [ ]:
winlogsA=pd.read_parquet('part.88.parquet')

winlogs=winlogsA.sample(10000,replace=False)#.iloc[:,:3]
winlogs=winlogs.dropna(how='any',axis=1)

g = graphistry.nodes(cudf.from_pandas(winlogs))
# g=g.featurize(feature_engine='cu_cat',memoize=False)
g2=g.umap(feature_engine='cu_cat')

In [ ]:
g2.plot()

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n = [500]
for p in np.arange(X):

  for ii,i in enumerate(n):
    
    newsA, _ = fetch_20newsgroups(
        shuffle=True,
        random_state=1,
        remove=("headers", "footers", "quotes"),
        return_X_y=True,
    )

    news = newsA[:i]
    news=pd.DataFrame(news)
    # newsA=pd.DataFrame(newsA)
    
    g = graphistry.nodes(cudf.from_pandas(news))
    t=time()
    g2 = g.umap(feature_engine='cu_cat',memoize=False)
    t1=time()-t


    g = graphistry.nodes(news)
    t=time()
    g2 = g.umap(feature_engine='dirty_cat',memoize=False)
    t2=time()-t

    bench.loc[1+bench.shape[0]]=['20news',i,news.shape[1],t1,t2]
!cp *.txt /content/drive/MyDrive/graphistry

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))
for p in np.arange(X):

  t=time()
  g5 = graphistry.nodes((tdf))
  g5.umap(X=['feats'], feature_engine='cu_cat',memoize=False)
  t1=time()-t

  t=time()
  g5 = graphistry.nodes((tdf))
  g5.umap(X=['feats'], feature_engine='dirty_cat',memoize=False)
  t2=time()-t
  bench.loc[1+bench.shape[0]]=['redteam',i,news.shape[1],t1,t2]
  bench.to_csv('redteam_bench.txt',sep='\t',mode='a')
!cp *.txt /content/drive/MyDrive/graphistry

In [ ]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State','TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



In [ ]:
for p in np.arange(X):

  t=time()
  g = graphistry.edges(cudf.from_pandas(edf), src, dst)

  g1a = g.featurize(kind='edges', 
              X=good_cols_with_edges, 
              y = ['bot'], 
              # use_scaler='quantile',
              # use_scaler_target=None,
              cardinality_threshold=20,
              cardinality_threshold_target=2, 
              n_topics=n_topics,
              feature_engine='cu_cat',
              # engine='cuml',
              memoize=False,
              n_topics_target=n_topics_target,
              n_bins=n_topics_target,
              # metric='euclidean', 
              # n_neighbors=12)
                  )
  t1=time()-t

  t=time()
  g = graphistry.edges((edf), src, dst)

  g1a = g.featurize(kind='edges', 
              X=good_cols_with_edges, 
              y = ['bot'], 
              # use_scaler='quantile',
              # use_scaler_target=None,
              cardinality_threshold=20,
              cardinality_threshold_target=2, 
              n_topics=n_topics,
              feature_engine='dirty_cat',
              # engine='cuml',
              memoize=False,
              n_topics_target=n_topics_target,
              n_bins=n_topics_target,
              # metric='euclidean', 
              # n_neighbors=12)
                  )
  t2=time()-t

  bench.loc[bench.shape[0]+1]=['ctu13',i,news.shape[1],t1,t2]
  bench.to_csv('ctu_bench.txt',sep='\t',mode='a')
!cp *.txt /content/drive/MyDrive/graphistry

In [ ]:
dfA = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)
for p in np.arange(X):

  n=[1000,2000,2999] #,1000000]
  for ii,i in enumerate(n):
      df = dfA.sample(i,replace=False) # set smaller if you want to test a minibatch 

      g = graphistry.nodes(cudf.from_pandas(df))


      t=time()
      g2 = g.featurize(X=['title', 'text'], # the features to encode (can add/remove 'text', etc)
                      y=['score'], # for demonstrative purposes, we include a target -- though this one is not really conditioned on textual features in a straightforward way
                      model_name='msmarco-distilbert-base-v2', #'paraphrase-MiniLM-L6-v2', etc, from sbert/Huggingface, the text encoding model
                      min_words = 0, # when 0 forces all X=[..] as textually encoded, higher values would ascertain if a column is textual or not depending on average number of words per column
                      use_ngrams=False, # set to True if you want ngram features instead (does not make great plots but useful for other situations)
                      # use_scaler_target='zscale', # for regressive targets
                      # use_scaler=None, # there are many more settings see `g.featurize?` and `g.umap?` for further options
                      feature_engine='cu_cat',
                      memoize=False,    
                    )
      t1=time()-t
      
      
      t=time()
      g2 = g.featurize(X=['title', 'text'], # the features to encode (can add/remove 'text', etc)
                      y=['score'], # for demonstrative purposes, we include a target -- though this one is not really conditioned on textual features in a straightforward way
                      model_name='msmarco-distilbert-base-v2', #'paraphrase-MiniLM-L6-v2', etc, from sbert/Huggingface, the text encoding model
                      min_words = 0, # when 0 forces all X=[..] as textually encoded, higher values would ascertain if a column is textual or not depending on average number of words per column
                      use_ngrams=False, # set to True if you want ngram features instead (does not make great plots but useful for other situations)
                      # use_scaler_target='zscale', # for regressive targets
                      # use_scaler=None, # there are many more settings see `g.featurize?` and `g.umap?` for further options
                      feature_engine='dirty_cat',
                      memoize=False,    
                    )
      t2=time()-t
      
      bench.loc[bench.shape[0]+1]=['askHN',i,news.shape[1],t1,t2]
      bench.to_csv('askHN_bench.txt',sep='\t',mode='a')
  !cp *.txt /content/drive/MyDrive/graphistry

In [ ]:
# winlogsA=pd.read_parquet('part.88.parquet')
for p in np.arange(X):

  i=1000000

  winlogs=winlogsA.sample(i,replace=False)#.iloc[:,:3]
  winlogs=winlogs.dropna(how='any',axis=1)
  # winlogs.shape
  g = graphistry.nodes(cudf.from_pandas(winlogs))

  t=time()
  # for jj,j in enumerate(['cu_cat','dirty_cat']):
  g.featurize(feature_engine='cu_cat',memoize=False)
  t1=time()-t

  t2=np.nan
  bench.loc[ii]=['winlogs',winlogs.shape[0],winlogs.shape[1],t1,t2]
  bench.to_csv('1m_winlogs_bench.txt',sep='\t',mode='a')
!cp *.txt /content/drive/MyDrive/graphistry

In [ ]:
# bench.to_csv('bench.txt',sep='\t')

In [ ]:
!cp *.txt /content/drive/MyDrive/graphistry

# run as script

In [ ]:
def load_data():
    ### WINLOGS
    # !wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet

    winlogsA=pd.read_parquet('part.88.parquet')

    ### 20NEWS ###
    # from sklearn.datasets import fetch_20newsgroups
    # newsgroups = fetch_20newsgroups()#categories=categories)

    ### REDTEAM ###
    df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
    red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
    df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
    df['feats2'] = df.src_computer + ' ' + df.dst_computer
    ndf = df.drop_duplicates(subset=['feats'])
    tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
    tdf['node'] = range(len(tdf))


    ### CTU13 ###
    edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
    T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
    bot = edf[T]
    nbot = edf[~T]
    print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

    # let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
    negs = nbot.sample(10*len(bot))
    edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
    edf = edf.drop_duplicates()

    # some useful indicators for later that predict Botnet as Bool and Int
    Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

    # Later we will use and exploit any meaning shared between the labels in a latent distribution

    # add it to the dataframe
    edf['bot'] = Y

    # name some columns for edges and features
    src = 'SrcAddr'
    dst = 'DstAddr'
    good_cols_with_edges = ['Dur', 'Proto', 'Sport',
        'Dport', 'State','TotBytes', 'SrcBytes', src, dst]

    good_cols_without_edges = ['Dur', 'Proto', 'Sport',
        'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

    ## some encoding parameters
    n_topics = 20
    n_topics_target = 7

    ### ASKHN ###
    dfA = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

    # nn=[1000,2000,2999] #,1000000]
    return winlogsA,tdf,edf,dfA

In [ ]:
def run_bench(X,winlogsA,tdf,edf,dfA):
    nn = [500]
    import time
    timestr = time.strftime("%Y%m%d-%H%M%S")
    from time import time
    from sklearn.datasets import fetch_20newsgroups

    bench=pd.DataFrame(columns=['dataset','row','col','cc_time','dc_time'])
    for p in np.arange(X):
        n=[1000,5000,10000,50000,100000] #,1000000]
        for ii,i in enumerate(n):

            winlogs=winlogsA.sample(i,replace=False)#.iloc[:,:3]
            winlogs=winlogs.dropna(how='any',axis=1)
            # winlogs.shape
            g = graphistry.nodes(cudf.from_pandas(winlogs))

            t=time()
            # for jj,j in enumerate(['cu_cat','dirty_cat']):
            g.featurize(feature_engine='cu_cat',memoize=False)
            t1=time()-t

            g = graphistry.nodes(winlogs)
            t=time()
            # for jj,j in enumerate(['cu_cat','']):
            g.featurize(feature_engine='dirty_cat',memoize=False)
            t2=time()-t

            bench.loc[ii]=['winlogs',i,winlogs.shape[1],t1,t2]
            del g


        for ii,i in enumerate(nn):

            newsA, _ = fetch_20newsgroups(
                shuffle=True,
                random_state=1,
                remove=("headers", "footers", "quotes"),
                return_X_y=True,
            )

            news = newsA[:i]
            news=pd.DataFrame(news)
            # newsA=pd.DataFrame(newsA)

            g = graphistry.nodes(cudf.from_pandas(news))
            t=time()
            g.umap(feature_engine='cu_cat',memoize=False)
            t1=time()-t


            g = graphistry.nodes(news)
            t=time()
            g.umap(feature_engine='dirty_cat',memoize=False)
            t2=time()-t

            bench.loc[1+bench.shape[0]]=['20news',i,news.shape[1],t1,t2]
            del g


        t=time()
        g5 = graphistry.nodes((tdf))
        g5.umap(X=['feats'], feature_engine='cu_cat',memoize=False)
        t1=time()-t

        t=time()
        g5 = graphistry.nodes((tdf))
        g5.umap(X=['feats'], feature_engine='dirty_cat',memoize=False)
        t2=time()-t
        bench.loc[1+bench.shape[0]]=['redteam',i,news.shape[1],t1,t2]
        del g5


        src = 'SrcAddr'
        dst = 'DstAddr'
        good_cols_with_edges = ['Dur', 'Proto', 'Sport',
              'Dport', 'State','TotBytes', 'SrcBytes', src, dst]

        good_cols_without_edges = ['Dur', 'Proto', 'Sport',
              'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']
        n_topics = 20
        n_topics_target = 7

        t=time()
        g = graphistry.edges(cudf.from_pandas(edf), src, dst)

        g.featurize(kind='edges', 
                    X=good_cols_with_edges, 
                    y = ['bot'], 
                    # use_scaler='quantile',
                    # use_scaler_target=None,
                    cardinality_threshold=20,
                    cardinality_threshold_target=2, 
                    n_topics=n_topics,
                    feature_engine='cu_cat',
                    # engine='cuml',
                    memoize=False,
                    n_topics_target=n_topics_target,
                    n_bins=n_topics_target,
                    # metric='euclidean', 
                    # n_neighbors=12)
                        )
        t1=time()-t

        t=time()
        g = graphistry.edges((edf), src, dst)

        g.featurize(kind='edges', 
                    X=good_cols_with_edges, 
                    y = ['bot'], 
                    # use_scaler='quantile',
                    # use_scaler_target=None,
                    cardinality_threshold=20,
                    cardinality_threshold_target=2, 
                    n_topics=n_topics,
                    feature_engine='dirty_cat',
                    # engine='cuml',
                    memoize=False,
                    n_topics_target=n_topics_target,
                    n_bins=n_topics_target,
                    # metric='euclidean', 
                    # n_neighbors=12)
                        )
        t2=time()-t

        bench.loc[bench.shape[0]+1]=['ctu13',i,news.shape[1],t1,t2]
        del g

        nn=[1000,2000,2999] #,1000000]
        for ii,i in enumerate(nn):
            df = dfA.sample(i,replace=False) # set smaller if you want to test a minibatch 

            g = graphistry.nodes(cudf.from_pandas(df))


            t=time()
            g.featurize(X=['title', 'text'], # the features to encode (can add/remove 'text', etc)
                            y=['score'], # for demonstrative purposes, we include a target -- though this one is not really conditioned on textual features in a straightforward way
                            model_name='msmarco-distilbert-base-v2', #'paraphrase-MiniLM-L6-v2', etc, from sbert/Huggingface, the text encoding model
                            min_words = 0, # when 0 forces all X=[..] as textually encoded, higher values would ascertain if a column is textual or not depending on average number of words per column
                            use_ngrams=False, # set to True if you want ngram features instead (does not make great plots but useful for other situations)
                            # use_scaler_target='zscale', # for regressive targets
                            # use_scaler=None, # there are many more settings see `g.featurize?` and `g.umap?` for further options
                            feature_engine='cu_cat',
                            memoize=False,    
                          )
            t1=time()-t


            t=time()
            g.featurize(X=['title', 'text'], # the features to encode (can add/remove 'text', etc)
                            y=['score'], # for demonstrative purposes, we include a target -- though this one is not really conditioned on textual features in a straightforward way
                            model_name='msmarco-distilbert-base-v2', #'paraphrase-MiniLM-L6-v2', etc, from sbert/Huggingface, the text encoding model
                            min_words = 0, # when 0 forces all X=[..] as textually encoded, higher values would ascertain if a column is textual or not depending on average number of words per column
                            use_ngrams=False, # set to True if you want ngram features instead (does not make great plots but useful for other situations)
                            # use_scaler_target='zscale', # for regressive targets
                            # use_scaler=None, # there are many more settings see `g.featurize?` and `g.umap?` for further options
                            feature_engine='dirty_cat',
                            memoize=False,    
                          )
            t2=time()-t

            bench.loc[bench.shape[0]+1]=['askHN',i,news.shape[1],t1,t2]
            del g

        # winlogsA=pd.read_parquet('part.88.parquet')
        i=1000000

        winlogs=winlogsA.sample(i,replace=False)#.iloc[:,:3]
        winlogs=winlogs.dropna(how='any',axis=1)
        # winlogs.shape
        g = graphistry.nodes(cudf.from_pandas(winlogs))

        t=time()
        # for jj,j in enumerate(['cu_cat','dirty_cat']):
        g.featurize(feature_engine='cu_cat',memoize=False)
        t1=time()-t

        t2=np.nan
        bench.loc[ii]=['winlogs',winlogs.shape[0],winlogs.shape[1],t1,t2]

        bench.to_csv('bench'+timestr+'.txt',sep='\t',mode='a')
        del g

In [ ]:
winlogsA,tdf,edf,dfA=load_data()
run_bench(1,winlogsA,tdf,edf,dfA)